In [51]:
import numpy as np
import numpy.polynomial.legendre as leg
import dask.array as da
import fmks
from fmks.func import curry
import fmks.bases.legendre as legendre
from toolz.curried import pipe

@curry
def scaled_data(data, domain):
    return (2. * data - domain[0] - domain[1]) / (domain[1] - domain[0])


@curry
def coeff(states):
    return np.eye(len(states)) * (states + 0.5)


@curry
def leg_data(data, coeff):
    return leg.legval(data, coeff)


@curry
def discretize(data, states = np.arange(2), domain=(0, 1)):
    return pipe(data[...,None],
                scaled_data(domain = domain),
                leg_data(coeff = coeff(states)))

@curry
def legendre_basis(x_data, n_state=2, domain=(0, 1), chunks = (1,)):
    return (discretize(da.from_array(x_data, chunks = (1,) + x.shape[1:]),
                       np.arange(n_state),
                       domain),
            lambda x: slice(-1))

In [36]:
np.random.seed(101)

In [37]:
shape   = (3, 5, 5)
chunks  = (1, 5, 5)
domain  = (-1, 1)
n_state = 4

In [38]:
x = 2*np.random.random(shape)

In [39]:
x_ = legendre.legendre_basis(x, n_state, domain)
x_[0].shape

(3, 5, 5, 4)

In [52]:
 x_1 = legendre_basis(x, n_state, domain)
x_1[0].shape

(3, 5, 5, 4)

In [41]:
np.allclose(x_[0], x_1[0].compute())

True